## Import Dependencies

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import ee
import geemap
import os
from tqdm.auto import tqdm

data_dir = r'/beegfs/halder/jupyter_playgroundnoconda_p3.12_1/jupyter_work/project/GITHUB/Landscape-Analysis/data/VECTOR'
out_dir = r'/beegfs/halder/jupyter_playgroundnoconda_p3.12_1/jupyter_work/project/GITHUB/Landscape-Analysis/data/OUTPUT'

In [13]:
# ee.Authenticate()
ee.Initialize(project='ee-geonextgis')

In [14]:
Map = geemap.Map(basemap='Esri.WorldImagery')
# Map

In [15]:
# Read the ROI
roi_bbox_ee = ee.Geometry.Rectangle([5.86736702, 47.27011403, 15.04089846, 55.05719656]).buffer(10000)
roi_style = {'fillColor': '00000000', 'color': 'red', 'width': 1}
Map.addLayer(roi_bbox_ee, {}, 'ROI')
Map.centerObject(roi_bbox_ee, 7)

In [16]:
# Load Copernicus DEM
glo30 = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30")
glo30 = glo30.mosaic().setDefaultProjection('EPSG:25832', None, 30).clip(roi_bbox_ee).rename('Elevation')
glo30_slope = ee.Terrain.slope(glo30).rename('Slope')
glo30 = glo30.addBands(glo30_slope)

# Create a custom elevation palette from hex strings.
elevationPalette = ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff'];

# Use these visualization parameters, customized by location.
visParams_elev = {'min': 1, 'max': 3000, 'palette': elevationPalette};
visParams_slope = {'min': 0, 'max': 7, 'palette': elevationPalette};

Map.addLayer(glo30.select('Elevation'), visParams_elev, 'Elevation')
Map.addLayer(glo30.select('Slope'), visParams_slope, 'Slope')

In [23]:
# Load the Grids
grids_gdf = gpd.read_file(os.path.join(data_dir, 'DE_Hexbins_2.5sqkm_EPSG_25832.shp'))
grids_gdf = grids_gdf[['id', 'geometry']]
grids_gdf['id'] = grids_gdf['id'].astype('int')

print(grids_gdf.shape)
grids_gdf.head()

(67545, 2)


,id,geometry
0,68921,"POLYGON ((709129.96 5565059.119, 709851.648 55..."
1,68920,"POLYGON ((709129.96 5567559.119, 709851.648 55..."
2,68923,"POLYGON ((709129.96 5560059.119, 709851.648 55..."
3,68922,"POLYGON ((709129.96 5562559.119, 709851.648 55..."
4,68925,"POLYGON ((709129.96 5555059.119, 709851.648 55..."


In [26]:
# out_folder = os.path.join(out_dir, 'Landscape_Metrics_2.5KM', 'Topo_Stats')
# os.makedirs(out_folder, exist_ok=True)

# batch_size = 1000
# groups = range(0, len(grids_gdf), batch_size)

# for batch_id, group in tqdm(enumerate(groups), total=len(groups)):
#     out_dem_stats = os.path.join(out_folder, f'DEM_Stat_STD_{batch_id}.csv')
#     batch_gdf = grids_gdf.iloc[group:group+batch_size]
#     batch_gdf_ee = geemap.gdf_to_ee(batch_gdf)
#     geemap.zonal_statistics(glo30, batch_gdf_ee, out_dem_stats, statistics_type='STD', scale=30)